# Agreement on the taxonomy of ASAT documentation

## Package installation and loading

In [20]:
install.packages('irr')
install.packages('irrCAC')
install.packages('tidyverse')

Installation du package dans ‘/usr/local/lib/R/4.3/site-library’
(car ‘lib’ n'est pas spécifié)

Installation du package dans ‘/usr/local/lib/R/4.3/site-library’
(car ‘lib’ n'est pas spécifié)

Installation du package dans ‘/usr/local/lib/R/4.3/site-library’
(car ‘lib’ n'est pas spécifié)



In [21]:
library('irr')
library('irrCAC')
library('tidyverse')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


## Functions

Multi-label agreement formulae use the code of [https://gdmcdonald.github.io/multi-label-inter-rater-agreement/Multi-Label_Agreement.html](https://gdmcdonald.github.io/multi-label-inter-rater-agreement/Multi-Label_Agreement.html)

In [14]:
#' Parse string into a character vector
#'
#' @param x string, e.g. "label_1, label_2"
#' @param sep separator, e.g. ", "
#'
#' @return character vector of labels, e.g. c("label_1", "label_2")
#' @export
#'
#' @examples
#' elements_from_string("l1, l2, l3", sep = ", ")
elements_from_string <- function(x, sep = ", ") {str_split(x,sep,simplify = F)[[1]]}

#' Measuring Agreement on Set-valued Items (MASI) distance from text string
#' MASI Similarity or Distance (pairwise)
#'
#' @param x Person x string of labels such as "label_1, label_2, label_3"
#' @param y Person y string of labels such as "label_4, label_1, label_5, label_7"
#' @param sep Label separator in the string, default = ", "
#' @param jaccard_only Only return Jaccard index instead of MASI (default = FALSE)
#' @param type one of "dist" or "sim" (default) for a distance or similarity score.
#'
#' @return Jaccard Distance between the two sets
#' @export
#'
#' @examples
#' masi("l1, l2, l3", "l7, l2")
masi <- function(x,y,sep = ", ", jaccard_only = F, type = "sim"){
  # Define the labels for each rater
  lab_x <- elements_from_string(x)
  lab_y <- elements_from_string(y)
  
  # compute set diff and intersection size
  diff_xy_size <- length(setdiff(lab_x,lab_y)) # number of elements in set x but not in set y
  diff_yx_size <- length(setdiff(lab_y,lab_x)) # number of elements in set y but not in set x
  intersection_size <- length(intersect(lab_x,lab_y)) # number of elements in common between two sets
  
  # monotonicity simillarity coefficient, M, see http://www.lrec-conf.org/proceedings/lrec2006/pdf/636_pdf.pdf Rebecca Passonneau. 2006. Measuring Agreement on Set-valued Items (MASI) for Semantic and Pragmatic Annotation. In Proceedings of the Fifth International Conference on Language Resources and Evaluation (LREC’06), Genoa, Italy. European Language Resources Association (ELRA).
  m_sim <- case_when(
    (diff_xy_size == 0) & (diff_yx_size == 0) ~ 1, # the sets are identical, return 1
    (diff_xy_size == 0) | (diff_yx_size == 0) ~ 2/3, # one set is a subset of the other, return 2/3
    (diff_xy_size != 0) & (diff_yx_size != 0) & (intersection_size !=0) ~ 1/3, # some overlap, some non-overlap in each set, return 1/3
    intersection_size ==0 ~ 0 # disjoint sets, return 0
  )
  
  # Calculate Jaccard simmilarity; J=1 means same, J=0 means no overlap at all. See https://en.wikipedia.org/wiki/Jaccard_index
  jaccard_sim <- intersection_size/(length(lab_x) + length(lab_y) - intersection_size)
  
  #MASI sim is M*J; MASI dist is 1-M*J
  masi_sim <- if_else(jaccard_only,
                      jaccard_sim,
                      m_sim*jaccard_sim)
  
  return(if_else(type == "sim",
                 masi_sim,
                 1-masi_sim))
}


MASI_simmilarity_matrix <- function(df, sep = ", ") {
  labels_all_combos <- sort(unique(unlist(df))) # alphabetical sorted list of all strings of labels
  
  num_label_combos <- length(labels_all_combos) # number of combinations above
  
  masi_sim_mat <- matrix(nrow = num_label_combos,
                         ncol = num_label_combos,
                         dimnames = list(labels_all_combos,
                                         labels_all_combos))
  
  for(i in 1:num_label_combos){
    for(j in 1:num_label_combos)
    {
      masi_sim_mat[i,j] <- masi(x = labels_all_combos[i],
                                y = labels_all_combos[j],
                                sep = sep)
    }}
  
  return(masi_sim_mat)
}

In [15]:
ml_fleiss_kappa <- function(dataset, sep=",") {
    wt <- MASI_simmilarity_matrix(dataset, sep=sep)
    result <- fleiss.kappa.raw(ratings = dataset, 
                        weights = wt,
                        categ.labels = rownames(wt),
                        conflev = 0.95)
    return(result)
}

## Agreement on text coding

In [11]:
text_r0_r1 <- read.csv("text-r0-r1.csv", sep=";")
text_r0_r2 <- read.csv("text-r0-r2.csv", sep=";")
text_r0_r3 <- read.csv("text-r0-r3.csv", sep=";")

In [6]:
kappa2(text_r0_r1[, c("r0", "r1")], weight = "unweighted")

 Cohen's Kappa for 2 Raters (Weights: unweighted)

 Subjects = 278 
   Raters = 2 
    Kappa = 0.299 

        z = 7.85 
  p-value = 4e-15 

In [9]:
kappa2(text_r0_r2[, c("r0", "r2")], weight = "unweighted")

 Cohen's Kappa for 2 Raters (Weights: unweighted)

 Subjects = 185 
   Raters = 2 
    Kappa = 1 

        z = 18.2 
  p-value = 0 

In [12]:
kappa2(text_r0_r3[, c("r0", "r3")], weight = "unweighted")

 Cohen's Kappa for 2 Raters (Weights: unweighted)

 Subjects = 428 
   Raters = 2 
    Kappa = 0.741 

        z = 21.4 
  p-value = 0 

## Agreement on code coding

In [34]:
code_r0_r1 <- read.csv("code-r0-r1.csv", sep=";")
code_r0_r2 <- read.csv("code-r0-r2.csv", sep=";")
code_r0_r3 <- read.csv("code-r0-r3.csv", sep=";")

In [30]:
ml_fleiss_kappa(code_r0_r1[, c("r0", "r1")], sep=",")

$est
     coeff.name pa      pe coeff.val coeff.se conf.int p.value         w.name
1 Fleiss' Kappa  1 0.53125         1        0    (1,1)       0 Custom Weights

$weights
     Fix What
Fix    1    0
What   0    1

$categories
[1] "FIX"  "WHAT"

In [31]:
ml_fleiss_kappa(code_r0_r2[, c("r0", "r2")], sep=",")

$est
     coeff.name        pa        pe coeff.val coeff.se   conf.int   p.value
1 Fleiss' Kappa 0.8666667 0.5466667   0.70588  0.30991 (-0.155,1) 0.0850041
          w.name
1 Custom Weights

$weights
                Fix      What What, Fix
Fix       1.0000000 0.0000000 0.3333333
What      0.0000000 1.0000000 0.3333333
What, Fix 0.3333333 0.3333333 1.0000000

$categories
[1] "FIX"       "WHAT"      "WHAT, FIX"

In [35]:
ml_fleiss_kappa(code_r0_r3[, c("r0", "r3")], sep=",")

$est
     coeff.name pa      pe coeff.val coeff.se conf.int p.value         w.name
1 Fleiss' Kappa  1 0.53125         1        0    (1,1)       0 Custom Weights

$weights
     Fix What
Fix    1    0
What   0    1

$categories
[1] "FIX"  "WHAT"

## Agreement on link coding

In [43]:
link_r0_r1 <- read.csv("link-r0-r1.csv", sep=";")
link_r0_r2 <- read.csv("link-r0-r2.csv", sep=";")
link_r0_r3 <- read.csv("link-r0-r3.csv", sep=";")

In [38]:
ml_fleiss_kappa(link_r0_r1[, c("r0", "r1")], sep=",")

$est
     coeff.name        pa       pe coeff.val coeff.se       conf.int   p.value
1 Fleiss' Kappa 0.4351852 0.412037   0.03937  0.20818 (-0.419,0.498) 0.8534469
          w.name
1 Custom Weights

$weights
                     Fix      What What, Why What, Why, Fix       Why
Fix            1.0000000 0.0000000 0.0000000      0.2222222 0.0000000
What           0.0000000 1.0000000 0.3333333      0.2222222 0.0000000
What, Why      0.0000000 0.3333333 1.0000000      0.4444444 0.3333333
What, Why, Fix 0.2222222 0.2222222 0.4444444      1.0000000 0.2222222
Why            0.0000000 0.0000000 0.3333333      0.2222222 1.0000000

$categories
[1] "FIX"            "WHAT"           "WHAT, WHY"      "WHAT, WHY, FIX"
[5] "WHY"

In [41]:
ml_fleiss_kappa(link_r0_r2[, c("r0", "r2")], sep=",")

$est
     coeff.name        pa        pe coeff.val coeff.se  conf.int   p.value
1 Fleiss' Kappa 0.5833333 0.3755556   0.33274  0.35767 (-0.66,1) 0.4048752
          w.name
1 Custom Weights

$weights
                       Fix      What What, Why What, Why, Fix       Why
               1 0.0000000 0.0000000 0.0000000      0.0000000 0.0000000
Fix            0 1.0000000 0.0000000 0.0000000      0.2222222 0.0000000
What           0 0.0000000 1.0000000 0.3333333      0.2222222 0.0000000
What, Why      0 0.0000000 0.3333333 1.0000000      0.4444444 0.3333333
What, Why, Fix 0 0.2222222 0.2222222 0.4444444      1.0000000 0.2222222
Why            0 0.0000000 0.0000000 0.3333333      0.2222222 1.0000000

$categories
[1] ""               "FIX"            "WHAT"           "WHAT, WHY"     
[5] "WHAT, WHY, FIX" "WHY"

In [44]:
ml_fleiss_kappa(link_r0_r3[, c("r0", "r3")], sep=",")

$est
     coeff.name        pa    pe coeff.val coeff.se   conf.int   p.value
1 Fleiss' Kappa 0.5333333 0.455   0.14373  0.43873 (-0.849,1) 0.7507023
          w.name
1 Custom Weights

$weights
                  Fix      What What, Fix What, Why
          1 0.0000000 0.0000000 0.0000000 0.0000000
Fix       0 1.0000000 0.0000000 0.3333333 0.0000000
What      0 0.0000000 1.0000000 0.3333333 0.3333333
What, Fix 0 0.3333333 0.3333333 1.0000000 0.1111111
What, Why 0 0.0000000 0.3333333 0.1111111 1.0000000

$categories
[1] ""          "FIX"       "WHAT"      "WHAT, FIX" "WHAT, WHY"